In [1]:
import pandas as pd
import numpy as np

In [10]:
#Global paramenters
#API URLs from RxNorm
base_url = "https://rxnav.nlm.nih.gov/REST/rxcui/"
base_url_2 = "https://rxnav.nlm.nih.gov/REST/"
base_url_3 = "https://rxnav.nlm.nih.gov/REST/rxcui.json?name="
#RxNorm methods
getAllRelatedInfo = "/allrelated.json"
getRelatedByType = "/related.json?tty=IN"
filterVet = "/filter.json?propName=RXNAV_VET_DRUG"
filterHuman = "/filter.json?propName=RXNAV_HUMAN_DRUG"
#dataframe
df = pd.DataFrame()

In [3]:
#Method to get drug with only the opioid ingredient
def query_self(name_of_drug):
    rd_df = pd.read_json(base_url+name_of_drug+getRelatedByType)
    rd_df = pd.DataFrame(rd_df.relatedGroup.conceptGroup)
    rd_df = pd.DataFrame(rd_df.conceptProperties[0])
    return rd_df
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+name_of_drug+getAllRelatedInfo)
    rd_df = pd.DataFrame(rd_df.allRelatedGroup.conceptGroup)
    rd_df = rd_df.loc[rd_df["tty"] != "DF"].reset_index(drop=True)
    rd_df = rd_df.loc[rd_df["tty"] != "DFG"].reset_index(drop=True)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
            row += 1
    return df

ALL Opioid Ingredients

In [4]:
#Read in ingredients from excel file
drug_ingredients = pd.read_excel("opioid_Ingredients.xlsx", header=None)
drugs = []
#Get rxcui for each ingredient
for drug in drug_ingredients[0]:
    print(drug)
    ingredient = pd.read_json(base_url_3+drug)
    drugs.append(ingredient.idGroup.rxnormId[0])
#loop array to get complete list of drugs
df_opioid = pd.DataFrame()
for drug in drugs:
    self_df = query_self(drug)
    ingredient = self_df["name"].values[0]
    self_df["ingredient"] = ingredient
    merg_df = query_drug(drug)
    merg_df["ingredient"] = ingredient
    df_opioid = pd.concat([df_opioid, self_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
    df_opioid = pd.concat([df_opioid, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_opioid

alfentanil
alphaprodine
Anileridine
benzhydrocodone
buprenorphine
butorphanol
codeine
dextromoramide
dezocine
difenoxin
dihydrocodeine
diphenoxylate
dipipanone
ethylmorphine
fentanyl
heroin
hydrocodone
hydromorphone
levomethadone
levomethadyl
levopropoxyphene
levorphanol
meperidine
meptazinol
methadone
methadyl+acetate
morphine
nalbuphine
normethadone
noscapine
oliceridine
opium
oxycodone
oxymorphone
papaveretum
paregoric
pentazocine
pethidine
phenazocine
phenoperidine
Pholcodine
pirinitramide
propoxyphene
remifentanil
sufentanil
tapentadol
tetrapeptide-15
tilidine
tramadol


,rxcui,name,synonym,tty,language,suppress,umlscui,ingredient
0,480,alfentanil,,IN,ENG,N,,alfentanil
1,203203,alfentanil hydrochloride,,PIN,ENG,N,,alfentanil
2,106500,alfentanil 5 MG/ML Injectable Solution,alfentanil (as alfentanil hydrochloride) 5 MG/...,SCD,ENG,N,,alfentanil
3,1723206,2 ML alfentanil 0.5 MG/ML Injection,alfentanil (as alfentanil HCl) 500 MCG/ML in 2...,SCD,ENG,N,,alfentanil
4,1723208,10 ML alfentanil 0.5 MG/ML Injection,alfentanil (as alfentanil HCl) 500 MCG/ML in 1...,SCD,ENG,N,,alfentanil
...,...,...,...,...,...,...,...,...
2730,1157662,tramadol Oral Product,,SCDG,ENG,N,,tramadol
2731,1157663,tramadol Pill,,SCDG,ENG,N,,tramadol
2732,1295674,tramadol Disintegrating Oral Product,,SCDG,ENG,N,,tramadol
2733,2588475,celecoxib / tramadol Oral Product,,SCDG,ENG,N,,tramadol


In [41]:
for rxcui in df_opioid.rxcui:
    rs = pd.read_json(base_url+rxcui+filterVet, typ="series")
    if rs.empty is False:
        rs2 = pd.read_json(base_url+rxcui+filterHuman)
        if rs2.empty 
rs

NOO
NOO
NOO
NOO


KeyboardInterrupt: 

In [22]:
rxcui

'1819'

In [34]:
newData = pd.read_json(base_url+"1819"+filterVet, typ="series")

In [40]:
newData[0]

1819

In [33]:
newData.to_frame('rxcui')

,rxcui
rxcui,1819


In [20]:
rs

""


In [17]:
rs.empty

True

In [10]:
#Reduce to usefull columns
df_opioid = df_opioid[['rxcui', 'name',"tty", "ingredient"]]
df_opioid

,rxcui,name,tty,ingredient
0,480,alfentanil,IN,alfentanil
1,479,Alfenta,BN,alfentanil
2,203203,alfentanil hydrochloride,PIN,alfentanil
3,564334,alfentanil 0.5 MG/ML [Alfenta],SBDC,alfentanil
4,1723217,alfentanil Injection [Alfenta],SBDF,alfentanil
...,...,...,...,...
2606,1157662,tramadol Oral Product,SCDG,tramadol
2607,1157663,tramadol Pill,SCDG,tramadol
2608,1295674,tramadol Disintegrating Oral Product,SCDG,tramadol
2609,2588475,celecoxib / tramadol Oral Product,SCDG,tramadol


In [11]:
#Export
df_opioid.to_excel("New_Complete_Opioids_List.xlsx", index=False)

In [12]:
#take list and format it to the RXNorm api format
#This list was made by combinding the excel files
df_list = pd.read_excel("Complete_Opioids_List.xlsx")
df_list = df_list[['Code', 'Description','TTY']]
df_list.rename(columns= {'Code' : 'rxcui', 'Description': 'name'}, inplace = True)
df_list

,rxcui,name,TTY
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SCD
1,1010603,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SBD
2,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SCD
3,1010606,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SBD
4,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,SCD
...,...,...,...
2254,995842,Biotussin,BN
2255,2179635,tramadol hydrochloride 100 MG Oral Tablet,SCD
2256,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD
2257,864826,methadone hydrochloride 5 MG/ML Oral Solution,SCD


In [13]:
df_list["ingredients"] = None
for i, row in df_list.iterrows():
    try:
        df_active_ing = query_self(str(df_list.loc[i, "rxcui"]))
        df_list.at[i, 'ingredients'] = list(df_active_ing['name'])
    except AttributeError:
        continue

In [14]:
df_list['ingredients'] = df_list['ingredients'].fillna('Obsolete Error')
df_list

,rxcui,name,TTY,ingredients
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SCD,"[buprenorphine, naloxone]"
1,1010603,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SBD,"[buprenorphine, naloxone]"
2,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SCD,"[buprenorphine, naloxone]"
3,1010606,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SBD,"[buprenorphine, naloxone]"
4,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,SCD,"[acetaminophen, oxycodone]"
...,...,...,...,...
2254,995842,Biotussin,BN,"[codeine, guaifenesin, pseudoephedrine]"
2255,2179635,tramadol hydrochloride 100 MG Oral Tablet,SCD,[tramadol]
2256,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete Error
2257,864826,methadone hydrochloride 5 MG/ML Oral Solution,SCD,[methadone]


In [15]:
df_list

,rxcui,name,TTY,ingredients
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SCD,"[buprenorphine, naloxone]"
1,1010603,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,SBD,"[buprenorphine, naloxone]"
2,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SCD,"[buprenorphine, naloxone]"
3,1010606,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,SBD,"[buprenorphine, naloxone]"
4,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,SCD,"[acetaminophen, oxycodone]"
...,...,...,...,...
2254,995842,Biotussin,BN,"[codeine, guaifenesin, pseudoephedrine]"
2255,2179635,tramadol hydrochloride 100 MG Oral Tablet,SCD,[tramadol]
2256,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete Error
2257,864826,methadone hydrochloride 5 MG/ML Oral Solution,SCD,[methadone]


In [16]:
df_list.to_excel("Complete_Opioids_List_with_Ingredients.xlsx", index=False)

In [17]:
#Identify new drugs found
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame()
new_drug = pd.DataFrame()
for code in df_opioid['rxcui']:
    drug_name = df_opioid['name'].where(df_opioid['rxcui'] == code).dropna().values[0]
    if df_list.isin([int(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = pd.concat([same_drug, df_opioid.loc[df_opioid['rxcui']==code]]).reset_index(drop=True)
    else:
        new_drug_count += 1
        new_drug = pd.concat([new_drug, df_opioid.loc[df_opioid['rxcui']==code]]).reset_index(drop=True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  1971
New drug count:  640


In [18]:
#export
same_drug.to_excel("Same_Opioid_Medications.xlsx", index=False)
new_drug.to_excel("New_Opioids_Added.xlsx", index=False)

In [19]:
absent_drugs = pd.DataFrame()
count = 0
#for every code from the combind excel list
for code in df_list['rxcui']:
    if df_opioid.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = pd.concat([absent_drugs, df_list.loc[df_list['rxcui']==code]]).reset_index(drop=True)
print('Count: ', count)
absent_drugs

Count:  1971


,rxcui,name,TTY,ingredients
0,1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD,Obsolete Error
1,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...,SBD,Obsolete Error
2,1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD,Obsolete Error
3,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD,Obsolete Error
4,1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD,Obsolete Error
...,...,...,...,...
283,71225,apomorphine hydrochloride,PIN,[apomorphine]
284,89781,"belladonna extract, USP",IN,"[belladonna extract, USP]"
285,993764,Capital and Codeine,BN,Obsolete Error
286,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete Error


In [20]:
#find the status of the not found drugs
df_status = pd.DataFrame()
index = 0
for code in absent_drugs['rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url_2+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    df_status = pd.concat([df_status, df_type]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_status

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
0,Obsolete,RXNORM,012011,,NO,012011,122021,,1050409
1,Obsolete,RXNORM,022011,,NO,022011,022023,,1087389
2,Obsolete,RXNORM,072011,,NO,072011,052021,,1112224
3,Obsolete,RXNORM,082011,,NO,082011,122021,,1113998
4,Obsolete,RXNORM,082011,,NO,082011,122021,,1114002
...,...,...,...,...,...,...,...,...,...
283,Active,RXNORM,042005,,YES,042005,,,71225
284,Active,RXNORM,042005,,YES,042005,,,89781
285,Obsolete,RXNORM,072010,,NO,072010,072021,,993764
286,Obsolete,RXNORM,092019,,NO,092019,122021,,2182349


In [21]:
df_not_found = absent_drugs.merge(df_status, on='rxcui')

In [22]:
df_not_found = df_not_found[['rxcui', 'name','TTY', 'status', 'ingredients']]
df_not_found

,rxcui,name,TTY,status,ingredients
0,1050409,oxycodone hydrochloride 20 MG/ML Oral Solution...,SBD,Obsolete,Obsolete Error
1,1087389,12 HR chlorpheniramine polistirex 4 MG / hydro...,SBD,Obsolete,Obsolete Error
2,1112224,chlorpheniramine maleate 0.8 MG/ML / hydrocodo...,SBD,Obsolete,Obsolete Error
3,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SCD,Obsolete,Obsolete Error
4,1114002,chlorpheniramine maleate 0.4 MG/ML / codeine p...,SBD,Obsolete,Obsolete Error
...,...,...,...,...,...
283,71225,apomorphine hydrochloride,PIN,Active,[apomorphine]
284,89781,"belladonna extract, USP",IN,Active,"[belladonna extract, USP]"
285,993764,Capital and Codeine,BN,Obsolete,Obsolete Error
286,2182349,chlorpheniramine maleate 0.4 MG/ML / hydrocodo...,SCD,Obsolete,Obsolete Error


In [23]:
df_active=df_not_found.loc[df_not_found['status']== 'Active'].reset_index(drop=True)
df_obsolete = df_not_found.loc[df_not_found['status']== 'Obsolete'].reset_index(drop=True)

In [24]:
#export
df_not_found.to_excel("Status_of_Not_found.xlsx", index=False)
df_active.to_excel("Active_Not_Found.xlsx", index=False)
df_obsolete.to_excel("Obsolete_Not_Found.xlsx", index=False)